In [ ]:
import pandas as pd
data=pd.read_csv('/content/drive/MyDrive/NLP DATASETS/IMDB Dataset.csv',index_col=None)
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
data['sentiment'].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [ ]:
sdata=data.sample(n=4000,random_state=1)
sdata.head()

,review,sentiment
26247,With No Dead Heroes you get stupid lines like ...,negative
35067,I thought maybe... maybe this could be good. A...,negative
34590,An elite American military team which of cours...,negative
16668,Ridiculous horror film about a wealthy man (Jo...,negative
12196,"Well, if you are one of those Katana's film-nu...",positive


In [ ]:
import re
def cleaning(text):
# to remove special characters
  text = re.sub(r'[^\w\s]', '',
str(text).lower().strip())
# to remove single character words
  text = re.sub(r"\b[a-zA-Z]\b", " ", text)
  text = re.sub(r"\n"," ",text)
  text = re.sub(r"\d+"," ",text)
  return text

In [ ]:
sdata["text_clean"] = sdata["review"].apply(lambda
x:cleaning(x))

In [ ]:
import spacy
spacyenglish = spacy.load("en_core_web_sm")
def lemmatizing(text):
  text=spacyenglish(text)
# getting lemma of each word
  lst_text = " ".join([word.lemma_ for word in text if
len(word)>2])
  return lst_text

In [ ]:
sdata['lemmatized_text']=sdata['text_clean'].apply(lambda
x:lemmatizing(x))

In [ ]:
#Stopwords removal
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stop_words = set(stopwords.words('english'))
stopwords=list(stop_words)

In [ ]:
def stopwords_remov(text):
  lst_text=text.split(" ")
  if stopwords is not None:
    for word in lst_text:
      lst_text = [word for word in lst_text if word not in stopwords]
# finally joing all words to make again string
  text=" ".join(lst_text)
  return text

In [ ]:
sdata['stopwords_removed']=sdata['lemmatized_text'].apply(lambda x:stopwords_remov(x))
sdata.head()

,review,sentiment,text_clean,lemmatized_text,stopwords_removed
26247,With No Dead Heroes you get stupid lines like ...,negative,with no dead heroes you get stupid lines like ...,with dead hero you get stupid line like that t...,dead hero get stupid line like woefully abysma...
35067,I thought maybe... maybe this could be good. A...,negative,thought maybe maybe this could be good an ea...,think maybe maybe this could good early appear...,think maybe maybe could good early appearance ...
34590,An elite American military team which of cours...,negative,an elite american military team which of cours...,elite american military team which course happ...,elite american military team course happen inc...
16668,Ridiculous horror film about a wealthy man (Jo...,negative,ridiculous horror film about wealthy man joh...,ridiculous horror film about wealthy man john ...,ridiculous horror film wealthy man john carrad...
12196,"Well, if you are one of those Katana's film-nu...",positive,well if you are one of those katanas filmnuts ...,well you be one those katanas filmnut just lik...,well one katanas filmnut like sure appreciate ...


In [ ]:
sdata_1=sdata[['stopwords_removed','sentiment']]

In [ ]:
sdata_1['sentiment'] = sdata_1.sentiment.map({'positive':1, 'negative':0})
sdata_1.head()

<ipython-input-30-ff95f1bd146b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdata_1['sentiment'] = sdata_1.sentiment.map({'positive':1, 'negative':0})


,stopwords_removed,sentiment
26247,dead hero get stupid line like woefully abysma...,0
35067,think maybe maybe could good early appearance ...,0
34590,elite american military team course happen inc...,0
16668,ridiculous horror film wealthy man john carrad...,0
12196,well one katanas filmnut like sure appreciate ...,1


In [ ]:
X=sdata_1['stopwords_removed']
y=sdata_1['sentiment']

**Count Vectorizer**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X_cvvector=cv.fit_transform(X)

In [ ]:
X_cvvector.shape

(4000, 36850)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_cvvector, y, test_size=0.2,
random_state=42)

In [ ]:
X_train=X_train.toarray()
X_test=X_test.toarray()

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(3200, 36850)
(3200,)
(800, 36850)
(800,)


**NaiveBais**

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
%time nb.fit(X_train, y_train)

CPU times: user 1.61 s, sys: 808 ms, total: 2.42 s
Wall time: 2.42 s


GaussianNB()

In [ ]:
y_pred_class = nb.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
print("Accuracy: ", accuracy_score(y_test, y_pred_class))

Accuracy:  0.61625


In [ ]:
from sklearn.metrics import classification_report
# Print the classification report
print(classification_report(y_test, y_pred_class))

              precision    recall  f1-score   support

    negative       0.62      0.66      0.64       412
    positive       0.61      0.57      0.59       388

    accuracy                           0.62       800
   macro avg       0.62      0.61      0.61       800
weighted avg       0.62      0.62      0.62       800



In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,y_pred_class)
print(cm)

[[271 141]
 [166 222]]


In [ ]:
d=pd.DataFrame()
d['y_test']=y_test
d['y_pred_class']=y_pred_class
d[d['y_test']!=d['y_pred_class']]

,y_test,y_pred_class
27099,negative,positive
2918,negative,positive
47085,positive,negative
4571,negative,positive
15709,positive,negative
...,...,...
24700,positive,negative
47745,positive,negative
34804,negative,positive
44648,negative,positive


**SVM**

In [ ]:
# prompt: implement support vector machine for above model

from sklearn.svm import SVC
svm = SVC()
%time svm.fit(X_train, y_train)
y_pred_class_svm = svm.predict(X_test)
print("Accuracy: ", accuracy_score(y_test, y_pred_class_svm))
# Print the classification report
print(classification_report(y_test, y_pred_class_svm))
cm_svm = confusion_matrix(y_test,y_pred_class_svm)
print(cm_svm)

CPU times: user 9min 14s, sys: 4.29 s, total: 9min 19s
Wall time: 6min 1s
Accuracy:  0.80375
              precision    recall  f1-score   support

    negative       0.87      0.73      0.79       412
    positive       0.75      0.89      0.81       388

    accuracy                           0.80       800
   macro avg       0.81      0.81      0.80       800
weighted avg       0.81      0.80      0.80       800

[[299 113]
 [ 44 344]]
